In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import pickle
import copy
import cv2
import math
from matplotlib import animation, rc
from IPython.display import HTML
%matplotlib inline

In [3]:
from opmap.RawCam import RawCam
from opmap.VmemMap import VmemMap
from opmap.PhaseMap import PhaseMap
from opmap.PhaseVarianceMap import PhaseVarianceMap
from opmap.PhaseVariancePeakMap import PhaseVariancePeakMap
from opmap.util import makeMovie

In [7]:
exp_dir = "/mnt/recordings/ExperimentData/"
anl_dir = "/mnt/recordings/AnalysisResult/Tomii/2017-07-19/"

df_path = os.path.join( anl_dir, "pv_stim.csv" )

delay = 5

pv_thre = 0.7

In [8]:
def show_session(s):
    for key in s.keys(): 
        print "{0}:{1}".format(key, s[key])
    
def integrate_all(cam, vmem, pmap, pvmap, stim_x, stim_y ):
    
    shape = pvmap.data.shape
    phase_wf = -1.9
    phase_wt = 2.3
    phase_w = 0.075
    step = vmem.data.shape[1] /  pvmap.data.shape[1] 
    
    rgb_img = np.zeros( (shape[0]-delay, shape[1], shape[2], 3), dtype=np.uint8)
    
    #------------------
    # Camera
    #------------------
    rgb_img[:,:,:,1] = (  (cam.data[delay:,::step,::step] - cam.vmin) / (cam.vmax-cam.vmin)*96 ).astype(np.uint8)
    
    #------------------
    # Vmem
    #------------------
    # rgb_img[:,:,:,0] = ( (vmem.data[delay:,::step,::step]-vmem.vmin)*0.5*255 ).astype(np.uint8)
    rgb_img[:,:,:,2] = 127 + ( vmem.data[delay:,::step,::step]*128 ).astype(np.uint8)
    rgb_img[:,:,:,0] = 127 + ( - vmem.data[delay:,::step,::step]*128 ).astype(np.uint8)
    
    #------------------
    # WF
    #------------------
    mask = (( np.abs( pmap.data - phase_wf ) < phase_w )*1.0).astype(np.uint8)[:-delay,:,:]
    for i in range(3) : rgb_img[:,:,:,i] *= (1-mask)
    rgb_img[:,:,:,2] += mask*255

    #------------------
    # WT
    #------------------
    mask = (( np.abs( pmap.data - phase_wt ) < phase_w )*1.0).astype(np.uint8)[:-delay,:,:]
    for i in range(3) : rgb_img[:,:,:,i] *= (1-mask)
    rgb_img[:,:,:,0] += mask*255
    
    #------------------
    # ROI
    #------------------
    for i in range(shape[0]-delay) : 
        for c in range(3):
            rgb_img[i,:,:,c] *= pvmap.roi[:,:].astype(np.uint8)
        
    #------------------
    # PS (#FFFFFF)
    #------------------
    mask = (( pvmap.data > pv_thre )*1.0).astype(np.uint8)[:-delay,:,:]
    for i in range(3) : rgb_img[:,:,:,i] *= (1-mask)
    for i in range(3) : rgb_img[:,:,:,i] += mask*(255*pvmap.data[:-delay,:,:]).astype(np.uint8)
        
    #------------------
    # Stim point (#FFFF00)
    #------------------
    mask = np.zeros( (shape[0]-delay, shape[1], shape[2]), dtype=np.uint8 )
    mask[ :, stim_y-1:stim_y+2, stim_x-1:stim_x+2 ] = 1
    for i in range(3) : rgb_img[:,:,:,i] *= (1-mask)
    rgb_img[:,:,:,2] += (mask*255).astype(np.uint8)
    rgb_img[:,:,:,1] += (mask*255).astype(np.uint8)        
    
    return rgb_img



## Data frameロード

In [9]:
df = pd.read_csv(df_path)
print "N:",len(df)
#df

N: 37


In [10]:
#sessions = range(len(df))

#sessions = range(5)

#sessions =  range(15, 25)

sessions = [4]

In [17]:
for i in sessions:
    
    s = df.loc[i,:]
    
    sample = s["sample"]
    session = s["session"]
    target = "{0}/{1}/C001H001S{2:0>4}".format(exp_dir, sample, session)    
    save_dir = target.replace(exp_dir, anl_dir)
    print "{0}:{1}".format(i, target)
    
    if 'cam' in locals() : del(cam)
    if 'vmem' in locals() : del(vmem)
    if 'pmap' in locals() : del(pmap)
    if 'pvmap' in locals() : del(pvmap)
    if 'pvpmap' in locals() : del(pvpmap)

    cam = RawCam(
        path = target,
        cam_type = 'sa4', 
        image_height = 512, image_width = 512, 
        frame_start = 0, frame_end = 500 )
    vmem = VmemMap( cam )
    vmem_ = copy.deepcopy(vmem)
    vmem_.setDiffRange(diff_min=64)
    vmem_.morphROI(closing=3)
    vmem_.smooth(9)
    pmap = PhaseMap( vmem_ )
    pvmap = PhaseVarianceMap(pmap, size=5)
    pvpmap = PhaseVariancePeakMap(pvmap)
    core_log = pvpmap.getCoreLog()
    rgb_img = integrate_all(cam, vmem_, pmap, pvmap, stim_x=s["stim_x"], stim_y=s["stim_y"])
    
    if True:
        np.savez(os.path.join(save_dir, "vmem"), vmem.data)
        np.savez(os.path.join(save_dir, "pmap"), pmap.data)
        np.savez(os.path.join(save_dir, "pvmap"), pvmap.data)
        np.savez(os.path.join(save_dir, "pvpmap"), pvpmap.data)

    with open(os.path.join(save_dir, "center.csv"), 'w') as outf:
        for row in core_log:
            str_line = ''
            for elem in row:
                str_line += '{0},'.format(elem)
            str_line.rstrip(',')
            outf.write(str_line)
            outf.write('\n')
    continue
            
    with open(os.path.join(save_dir, "elec_phase.log"), 'w') as outf:
        elec_phase = pmap.data[:, s["stim_y"], s["stim_x"]]
        for i, v in enumerate(elec_phase):
            outf.write('{0} {1}\n'.format(i, v))

    dirs = []

    #d = os.path.join( save_dir , "vmem");  vmem.saveImage(d); dirs.append(d)
    #d = os.path.join( save_dir , "pmap"); pmap.saveImage(d); dirs.append(d)
    #d = os.path.join( save_dir , "pvmap"); pvmap.saveImage(d); dirs.append(d)
    #d = os.path.join( save_dir , "pvpmap"); pvpmap.saveImage(d); dirs.append(d)

    d = os.path.join( save_dir , "all")
    if not os.path.exists(d) : os.mkdir(d)
    dirs.append(d)
    for i, img in enumerate(rgb_img):
        #im = plt.imshow(img)
        #plt.savefig(os.path.join(d, "{0:0>6}.png".format(i)))
        cv2.putText(img,"{0:0>4}ms".format(i), (4,16), cv2.FONT_HERSHEY_PLAIN, 0.6, (255,255,255))
        cv2.imwrite(os.path.join(d, "{0:0>6}.png".format(i)), img)
        
    for d in dirs: makeMovie(d)


IndentationError: expected an indented block (<ipython-input-17-5fbf15dc5966>, line 11)